In [12]:
from datasets import load_from_disk, Dataset, concatenate_datasets, ClassLabel, Features, Value, Sequence
from transformers import BertTokenizer, BertForSequenceClassification, BasicTokenizer
from torch.utils.data import  DataLoader
from tqdm.notebook import tqdm
import torch
import base
import os


In [13]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available and will be used:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("GPU is not available, using CPU.")

GPU is available and will be used: NVIDIA A100 80GB PCIe MIG 2g.20gb


In [14]:
augmentation_params = {"n_iter": 10, "p_mask":0.1, "p_pos": 0.3, "p_ng":0.2}

In [15]:
tokenizer = BasicTokenizer(do_lower_case=True)
DATASET = "sst2"

In [16]:
train_data = load_from_disk(f"~/data/{DATASET}/train")
sentences = list(map(lambda e: e["sentence"], train_data))

In [17]:
pos_tag_word_map_list = base.get_pos_tag_word_map(sentences, tokenizer=tokenizer)

In [18]:
base.reset_seed()

In [19]:
augmented_datasets = base.get_augmented_dataset(augmentation_params, train_data, pos_tag_word_map_list, tokenizer=tokenizer)

In [20]:
aug_datasets_formated = []
ds_schema = Features({
    "idx": Value("int32"),
    "sentence": Value("string"),
    "label": ClassLabel(names=["negative", "positive"])
})

for iter in augmented_datasets:
    dataset = Dataset.from_dict(iter)
    dataset = dataset.cast(ds_schema)
    aug_datasets_formated.append(dataset)

Casting the dataset:   0%|          | 0/53879 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/53879 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/53879 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/53879 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/53879 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/53879 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/53879 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/53879 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/53879 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/53879 [00:00<?, ? examples/s]

In [21]:
print(aug_datasets_formated[8][70])
print(train_data[70])

{'idx': 6575, 'sentence': '[MASK] reality it [MASK] s one tough rock .', 'label': 1}
{'idx': 6575, 'sentence': "in reality it 's one tough rock . ", 'label': 1}


In [22]:
tokenizer = BertTokenizer.from_pretrained("gchhablani/bert-base-cased-finetuned-sst2")
model = BertForSequenceClassification.from_pretrained("gchhablani/bert-base-cased-finetuned-sst2", num_labels=2)
model.to(device)
model.eval()

torch.save(model.state_dict(), f"{os.path.expanduser('~')}/models/{DATASET}/teacher.pth")

In [ ]:
train_dataset = base.prepare_dataset(train_data, tokenizer)
train_dataloader = DataLoader(train_dataset, batch_size=128, shuffle=False)
train_logits = base.generate_logits(train_dataloader, model)
train_dataset = train_dataset.add_column("logits", train_logits)
train_dataset = train_dataset.remove_columns(["token_type_ids", "attention_mask"])
train_dataset.set_format(type="torch", columns=["logits", "labels"], device="cpu")

Generating logits for given dataset:   0%|          | 0/421 [00:00<?, ?it/s]

In [24]:
print(base.check_acc(train_dataset, "Accuracy for base dataset: "))

Calculating accuracy based on the saved logits:   0%|          | 0/53879 [00:00<?, ?it/s]

Accuracy for base dataset:  0.9888268156424581


In [ ]:
aug_clean_datasets = []
for dataset in tqdm(aug_datasets_formated, total=(len(aug_datasets_formated)), desc="Processing augmented datasets: "):
    aug_train_dataset = base.prepare_dataset(dataset, tokenizer)
    aug_train_dataloader = DataLoader(aug_train_dataset, batch_size=128, shuffle=False)
    aug_train_logits = base.generate_logits(aug_train_dataloader, model)
    aug_train_dataset = aug_train_dataset.add_column("logits", aug_train_logits)
    aug_train_dataset = aug_train_dataset.remove_columns(["token_type_ids", "attention_mask"])
    aug_train_dataset.set_format(type="torch", columns=["logits", "labels"], device="cpu")

    aug_train_dataset = base.remove_diff_pred_class(train_dataset, aug_train_dataset, pytorch_dataset=False)
    
    aug_train_dataset.reset_format()
    aug_clean_datasets.extend(aug_train_dataset)

Processing augmented datasets:   0%|          | 0/10 [00:00<?, ?it/s]

Tokenizing the provided dataset:   0%|          | 0/53879 [00:00<?, ? examples/s]

Generating logits for given dataset:   0%|          | 0/421 [00:00<?, ?it/s]

Removing entries from augmented dataset that are different from the base one - based on saved logits:   0%|   …

Tokenizing the provided dataset:   0%|          | 0/53879 [00:00<?, ? examples/s]

...

Generating logits for given dataset:   0%|          | 0/421 [00:00<?, ?it/s]

Removing entries from augmented dataset that are different from the base one - based on saved logits:   0%|   …

Tokenizing the provided dataset:   0%|          | 0/53879 [00:00<?, ? examples/s]

Generating logits for given dataset:   0%|          | 0/421 [00:00<?, ?it/s]

Removing entries from augmented dataset that are different from the base one - based on saved logits:   0%|   …

In [26]:
train_dataset.reset_format()

In [27]:
print("First iter check")
print(aug_clean_datasets[1997])
filtered_base = train_dataset.filter(lambda x: x["idx"] == aug_clean_datasets[1997]["idx"])
print(filtered_base[0])

print("Second iter check")
print(aug_clean_datasets[66699])
filtered_base = train_dataset.filter(lambda x: x["idx"] == aug_clean_datasets[66699]["idx"])
print(filtered_base[0])


First iter check
{'idx': 20217, 'sentence': 'will be literally worth your time .', 'labels': 1, 'input_ids': [101, 1209, 1129, 6290, 3869, 1240, 1159, 119, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'logits': [-4.896175384521484, 4.807943344116211]}
{'idx': 20217, 'sentence': 'will be well worth your time . ', 'labels': 1, 'input_ids': [101, 1209, 1129, 1218, 3869, 1240, 1159, 119, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'logits': [-4.851616859436035, 4.748195171356201]}
Second iter check
{'idx': 30561, 'sentence': 'mr . man and [MASK] . [MASK] [MASK] strong and elysian performances , but', 'labels': 1, 'input_ids': [101, 182, 1197, 119, 1299, 1105, 103, 119, 103, 103, 2012, 1105, 8468, 6834, 1811, 3853, 117, 1133, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [28]:
print(train_dataset.features)

{'idx': Value(dtype='int32', id=None), 'sentence': Value(dtype='string', id=None), 'labels': ClassLabel(names=['negative', 'positive'], id=None), 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None), 'logits': Sequence(feature=Value(dtype='float32', id=None), length=-1, id=None)}


In [29]:
ds_schema = Features({
    "idx": Value("int32"),
    "sentence": Value("string"),
    "labels": ClassLabel(names=["negative", "positive"]),
    "input_ids": Sequence(feature=Value(dtype="int32")),
    "logits": Sequence(feature=Value(dtype="float32")),
})

aug_dataset = Dataset.from_list(aug_clean_datasets)
aug_dataset = aug_dataset.cast(ds_schema)


Casting the dataset:   0%|          | 0/479751 [00:00<?, ? examples/s]

In [30]:
aug_dataset.set_format(type="torch", columns=["logits", "labels"], device="cpu")
train_dataset.set_format(type="torch", columns=["logits", "labels"], device="cpu")

In [31]:
print(base.check_acc(train_dataset, "Accuracy for base dataset: "))
print(base.check_acc(aug_dataset, "Accuracy for augmented dataset: "))

Calculating accuracy based on the saved logits:   0%|          | 0/53879 [00:00<?, ?it/s]

Accuracy for base dataset:  0.9888268156424581


Calculating accuracy based on the saved logits:   0%|          | 0/479751 [00:00<?, ?it/s]

Accuracy for augmented dataset:  0.9904304524638823


In [32]:
train_all_data = concatenate_datasets([train_dataset, aug_dataset])
train_all_data.set_format(type="torch", columns=["logits", "labels"], device="cpu")

In [33]:
print(base.check_acc(train_all_data, "Accuracy for combined dataset: "))

Calculating accuracy based on the saved logits:   0%|          | 0/533630 [00:00<?, ?it/s]

Accuracy for combined dataset:  0.9902685381256676


In [34]:
print(train_all_data.column_names)

['idx', 'sentence', 'labels', 'input_ids', 'logits']


In [35]:
train_all_data.reset_format()
train_all_data = train_all_data.remove_columns(["idx", "input_ids"])

In [36]:
train_all_data.save_to_disk(f"~/data/{DATASET}/train-logits-augmented")

Saving the dataset (0/1 shards):   0%|          | 0/533630 [00:00<?, ? examples/s]

In [37]:
train_dataset.reset_format()
train_dataset = train_dataset.remove_columns(["idx", "input_ids"])
train_dataset.save_to_disk(f"~/data/{DATASET}/train-logits")

Saving the dataset (0/1 shards):   0%|          | 0/53879 [00:00<?, ? examples/s]

In [38]:
eval_data = load_from_disk(f"~/data/{DATASET}/eval")
eval_dataset = base.prepare_dataset(eval_data, tokenizer)
eval_dataloader = DataLoader(eval_dataset, batch_size=128, shuffle=False)

In [39]:
test_data = load_from_disk(f"~/data/{DATASET}/test")
test_dataset = base.prepare_dataset(test_data, tokenizer)
test_dataloader = DataLoader(test_dataset, batch_size=128, shuffle=False)

In [40]:
test_blank_data = load_from_disk(f"~/data/{DATASET}/test-blank")
test_blank_dataset = base.prepare_dataset(test_blank_data, tokenizer)
test_blank_dataloader = DataLoader(test_blank_dataset, batch_size=128, shuffle=False)

In [ ]:
eval_logits = base.generate_logits(eval_dataloader, model)
test_logits = base.generate_logits(test_dataloader, model)
test_blank_logits = base.generate_logits(test_blank_dataloader, model)

Generating logits for given dataset:   0%|          | 0/7 [00:00<?, ?it/s]

Generating logits for given dataset:   0%|          | 0/106 [00:00<?, ?it/s]

Generating logits for given dataset:   0%|          | 0/15 [00:00<?, ?it/s]

In [42]:
eval_dataset.reset_format()
eval_dataset = eval_dataset.add_column("logits", eval_logits)
eval_dataset = eval_dataset.remove_columns(["idx", "token_type_ids", "input_ids", "attention_mask"])

In [43]:
test_dataset.reset_format()
test_dataset = test_dataset.add_column("logits", test_logits)
test_dataset = test_dataset.remove_columns(["idx", "token_type_ids", "input_ids", "attention_mask"])

In [44]:
test_blank_dataset.reset_format()
test_blank_dataset = test_blank_dataset.add_column("logits", test_blank_logits)
test_blank_dataset = test_blank_dataset.remove_columns(["idx", "token_type_ids", "input_ids", "attention_mask"])

In [45]:
eval_dataset.save_to_disk(f"~/data/{DATASET}/eval-logits")
test_dataset.save_to_disk(f"~/data/{DATASET}/test-logits")
test_blank_dataset.save_to_disk(f"~/data/{DATASET}/test-blank-logits")

Saving the dataset (0/1 shards):   0%|          | 0/872 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/13470 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1821 [00:00<?, ? examples/s]

In [46]:
eval_data = load_from_disk(f"~/data/{DATASET}/eval-logits")
test_data = load_from_disk(f"~/data/{DATASET}/test-logits")

eval_data.set_format(type="torch", columns=["logits", "labels"], device="cpu")
test_data.set_format(type="torch", columns=["logits", "labels"], device="cpu")

print(base.check_acc(eval_data, "Accuracy for base eval dataset: "))
print(base.check_acc(test_data, "Accuracy for base test dataset: "))

Calculating accuracy based on the saved logits:   0%|          | 0/872 [00:00<?, ?it/s]

Accuracy for base eval dataset:  0.9231651376146789


Calculating accuracy based on the saved logits:   0%|          | 0/13470 [00:00<?, ?it/s]

Accuracy for base test dataset:  0.9896807720861173
